In [1]:
import pandas as pd
import os
import string
import unicodedata
from datetime import datetime
import math

In [15]:
df = pd.read_pickle('../../data/MICRODADOS.pkl')

In [16]:
# --------- FILTROS ---------
COLUMNS = ['DataCadastro', 'DataDiagnostico', 'DataEncerramento', 'Classificacao', 'Evolucao', 'Municipio']
#Municipios do ES
filtro_es = ['AGUIA BRANCA', 'ALTO RIO NOVO', 'ARACRUZ', 'BAIXO GUANDU',
  'COLATINA', 'GOVERNADOR LINDENBERG', 'IBIRACU', 'JOAO NEIVA',
  'LINHARES', 'MANTENOPOLIS', 'MARILANDIA', 'PANCAS', 'RIO BANANAL',
  'SAO DOMINGOS DO NORTE', 'SAO GABRIEL DA PALHA',
  'SAO ROQUE DO CANAA', 'SOORETAMA', 'VILA VALERIO',
  'AFONSO CLAUDIO', 'BREJETUBA', 'CARIACICA', 'CONCEICAO DO CASTELO',
  'DOMINGOS MARTINS', 'FUNDAO', 'GUARAPARI', 'IBATIBA', 'ITAGUACU',
  'ITARANA', 'LARANJA DA TERRA', 'MARECHAL FLORIANO',
  'SANTA LEOPOLDINA', 'SANTA MARIA DE JETIBA', 'SANTA TERESA',
  'SERRA', 'VENDA NOVA DO IMIGRANTE', 'VIANA', 'VILA VELHA',
  'VITORIA', 'AGUA DOCE DO NORTE', 'BARRA DE SAO FRANCISCO',
  'BOA ESPERANCA', 'CONCEICAO DA BARRA', 'ECOPORANGA', 'JAGUARE',
  'MONTANHA', 'MUCURICI', 'NOVA VENECIA', 'PEDRO CANARIO',
  'PINHEIROS', 'PONTO BELO', 'SAO MATEUS', 'VILA PAVAO', 'ALEGRE',
  'ALFREDO CHAVES', 'ANCHIETA', 'APIACA', 'ATILIO VIVACQUA',
  'BOM JESUS DO NORTE', 'CACHOEIRO DE ITAPEMIRIM', 'CASTELO',
  'DIVINO DE SAO LOURENCO', 'DORES DO RIO PRETO', 'GUACUI',
  'IBITIRAMA', 'ICONHA', 'IRUPI', 'ITAPEMIRIM', 'IUNA',
  'JERONIMO MONTEIRO', 'MARATAIZES', 'MIMOSO DO SUL', 'MUNIZ FREIRE',
  'MUQUI', 'PIUMA', 'PRESIDENTE KENNEDY', 'RIO NOVO DO SUL',
  'SAO JOSE DO CALCADO', 'VARGEM ALTA']

In [18]:
# --------- CRIANDO DF_CALENDAR_NEW(CASOS NOVOS) ---------
#df_calendar_new -> filtrar pacientes com covid confirmados; groupby(Municipio, DataDiagnostico); contar ocorrencias
df_calendar_new = df[df['Classificacao']=='Confirmados'].groupby(['Municipio','DataDiagnostico'])['DataCadastro'].size().reset_index(name='count_new')

#renomendo coluna DataDiagnostico
df_calendar_new.rename(columns={'DataDiagnostico': 'date'}, inplace=True)

#transformando o dtype na coluna 'date' para datetime
df_calendar_new['date'] = df_calendar_new['date'].astype('datetime64[ns]')

In [19]:
# --------- CRIANDO DF_CALENDAR_CLOSED(CASOS FECHADOS) ---------
#df_calendar_closed -> filtrar pacientes com covid confirmados; groupby(Municipio, DataEncerramento); contar ocorrencias
df_calendar_closed = df[df['Classificacao']=='Confirmados'].groupby(['Municipio','DataEncerramento'])['DataCadastro'].size().reset_index(name='count_closed')

#renomendo coluna DataEncerramento
df_calendar_closed.rename(columns={'DataEncerramento': 'date'}, inplace=True)

#transformando o dtype na coluna 'date' para datetime
df_calendar_closed['date'] = df_calendar_closed['date'].astype('datetime64[ns]')

In [20]:
# --------- CRIANDO DF_CALENDAR_OBITOS ---------
df_calendar_obitos = df[(df['Classificacao']=='Confirmados') & (df['Evolucao'] == 'Óbito pelo COVID-19')].groupby(['Municipio','DataEncerramento'])['DataCadastro'].size().reset_index(name='count_obito')

#renomendo coluna DataEncerramento
df_calendar_obitos.rename(columns={'DataEncerramento': 'date'}, inplace=True)

#transformando o dtype na coluna 'date' para datetime
df_calendar_obitos['date'] = df_calendar_obitos['date'].astype('datetime64[ns]')

In [21]:
# --------- MERGE ENTRE OS DFs CRIADOS ---------
df_calendar_temp = pd.merge(df_calendar_new, df_calendar_closed, how='outer', left_on=['Municipio', 'date'], right_on=['Municipio', 'date'])
df_calendar = pd.merge(df_calendar_temp, df_calendar_obitos, how='outer', left_on=['Municipio', 'date'], right_on=['Municipio', 'date'])


In [22]:
# --------- FILTRO DE DATAS ---------
#filtrando por período entre 2020 e 2021 (extrair dados com datas erradas)
df_calendar = df_calendar[(df_calendar['date'].dt.year>=2020) & (df_calendar['date'].dt.year<2022)]

In [23]:
# --------- DF DE DATAS AUXILIAR ---------
#criando df_date_aux para preencher o df_calendar com todas as datas do períodom, e fazendo o merge com o df_calendar por cidade
municipios = df_calendar['Municipio'].unique()

for i in municipios:
    delta = df_calendar[df_calendar['Municipio']==i]['date'].max() - df_calendar[df_calendar['Municipio']==i]['date'].min()
    df_date_aux = pd.DataFrame({'Municipio':i, 'date': pd.date_range(df_calendar[df_calendar['Municipio']==i]['date'].min(), periods=delta.days).tolist()})
    df_calendar = pd.merge(df_calendar, df_date_aux, how='outer', left_on=['Municipio', 'date'], right_on=['Municipio', 'date'])


In [24]:
# --------- AJUSTANDO DF_CALENDAR ---------
#organizando por cidade/data
df_calendar = df_calendar.sort_values(["Municipio", "date"]).reset_index()
df_calendar = df_calendar.drop('index', axis=1)

In [25]:
# --------- DEALING WITH NAN ---------
#preenchendo Nan com zero(0)
for i in ['count_new', 'count_closed', 'count_obito']:
    df_calendar[i] = df_calendar[i].fillna(0)

In [26]:
# --------- FEATURE ENGINEERING ---------
#criando coluna acumulado por cidade
municipios = df_calendar['Municipio'].unique()
acum = []
for idx, i in enumerate(df_calendar['date']):
    if df_calendar['Municipio'].iloc[idx] == df_calendar['Municipio'].iloc[idx-1]:
        try:
            acum.append(acum[idx-1] + df_calendar['count_new'].iloc[idx] - df_calendar['count_closed'].iloc[idx])
        except:
            acum.append(df_calendar['count_new'].iloc[idx] - df_calendar['count_closed'].iloc[idx])
    else:
        acum.append(0 + df_calendar['count_new'].iloc[idx] - df_calendar['count_closed'].iloc[idx])

df_calendar['acum'] = acum

In [27]:
#criando coluna casos recuperados por cidade
df_calendar['recup'] = df_calendar['count_closed'] - df_calendar['count_obito']


#criando colunas de dia/semana/dia_da_semana/mes/ano
df_calendar['day'] = [i.day for i in df_calendar['date']]
df_calendar['week'] = [i.week for i in df_calendar['date']]
df_calendar['weekday'] = [i.weekday() for i in df_calendar['date']]
df_calendar['month'] = [i.month for i in df_calendar['date']]
df_calendar['year'] = [i.year for i in df_calendar['date']]

In [28]:
#---------- CRIANDO COLUNAS PARA SEREM UTILIZADAS NO PLOT DE RESUMO ----------
cidades = df_calendar['Municipio'].unique()

#--- TOTAL CASOS FATAIS ACUMULADOS POR DIA POR CIDADE ---
fatais = []
for i in cidades:
    for j in range(len(df_calendar[df_calendar['Municipio']==i])):
        if j == 0:
            total = df_calendar[df_calendar['Municipio']==i]['count_obito'].iloc[j]
            fatais.append(total)
        else:
            total = total+df_calendar[df_calendar['Municipio']==i]['count_obito'].iloc[j]
            fatais.append(total)

df_calendar['fatais'] = fatais

In [29]:
%%time
#--- TOTAL CASOS CONFIRMADOS ACUMULADOS POR DIA POR CIDADE ---
confirmados = []
for i in cidades:
    for j in range(len(df_calendar[df_calendar['Municipio']==i])):
        if j == 0:
            total = df_calendar[df_calendar['Municipio']==i]['count_new'].iloc[j]
            confirmados.append(total)
        else:
            total = total+df_calendar[df_calendar['Municipio']==i]['count_new'].iloc[j]
            confirmados.append(total)

df_calendar['confirmados'] = confirmados

CPU times: user 12min 16s, sys: 3.72 s, total: 12min 20s
Wall time: 12min 44s


In [31]:
%%time
#--- TOTAL CASOS RECUPERADOS ACUMULADOS POR DIA POR CIDADE ---
recuperados = []
for i in cidades:
    for j in range(len(df_calendar[df_calendar['Municipio']==i])):
        if j == 0:
            total = df_calendar[df_calendar['Municipio']==i]['recup'].iloc[j]
            recuperados.append(total)
        else:
            total = total+df_calendar[df_calendar['Municipio']==i]['recup'].iloc[j]
            recuperados.append(total)

df_calendar['recuperados'] = recuperados

CPU times: user 12min 18s, sys: 3.95 s, total: 12min 22s
Wall time: 12min 37s


In [35]:
df_calendar.to_pickle('MICRODADOS_tratado.pkl')

In [33]:
%conda install fastparquet

Solving environment: done

## Package Plan ##

  environment location: /Users/dsa/opt/anaconda3

  added / updated specs:
    - fastparquet


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.3               |   py38hecd8cb5_0         2.9 MB
    fastparquet-0.5.0          |   py38he3068b8_1         172 KB
    thrift-0.11.0              |   py38h23ab428_0         102 KB
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  fastparquet        pkgs/main/osx-64::fastparquet-0.5.0-py38he3068b8_1
  thrift             pkgs/main/osx-64::thrift-0.11.0-py38h23ab428_0

The following packages will be UPDATED:

  conda                               4.10.1-py38hecd8cb5_1 --> 4.10.3-py38hecd8cb5_0



fastparquet-0.5.0    | 172 KB    | ###################################

In [30]:
def tratando_microdados():

    df = pd.read_pickle('../../data/MICRODADOS.pkl')
    
    # --------- FILTROS ---------
    COLUMNS = ['DataCadastro', 'DataDiagnostico', 'DataEncerramento', 'Classificacao', 'Evolucao', 'Municipio']
    #Municipios do ES
    filtro_es = ['AGUIA BRANCA', 'ALTO RIO NOVO', 'ARACRUZ', 'BAIXO GUANDU',
      'COLATINA', 'GOVERNADOR LINDENBERG', 'IBIRACU', 'JOAO NEIVA',
      'LINHARES', 'MANTENOPOLIS', 'MARILANDIA', 'PANCAS', 'RIO BANANAL',
      'SAO DOMINGOS DO NORTE', 'SAO GABRIEL DA PALHA',
      'SAO ROQUE DO CANAA', 'SOORETAMA', 'VILA VALERIO',
      'AFONSO CLAUDIO', 'BREJETUBA', 'CARIACICA', 'CONCEICAO DO CASTELO',
      'DOMINGOS MARTINS', 'FUNDAO', 'GUARAPARI', 'IBATIBA', 'ITAGUACU',
      'ITARANA', 'LARANJA DA TERRA', 'MARECHAL FLORIANO',
      'SANTA LEOPOLDINA', 'SANTA MARIA DE JETIBA', 'SANTA TERESA',
      'SERRA', 'VENDA NOVA DO IMIGRANTE', 'VIANA', 'VILA VELHA',
      'VITORIA', 'AGUA DOCE DO NORTE', 'BARRA DE SAO FRANCISCO',
      'BOA ESPERANCA', 'CONCEICAO DA BARRA', 'ECOPORANGA', 'JAGUARE',
      'MONTANHA', 'MUCURICI', 'NOVA VENECIA', 'PEDRO CANARIO',
      'PINHEIROS', 'PONTO BELO', 'SAO MATEUS', 'VILA PAVAO', 'ALEGRE',
      'ALFREDO CHAVES', 'ANCHIETA', 'APIACA', 'ATILIO VIVACQUA',
      'BOM JESUS DO NORTE', 'CACHOEIRO DE ITAPEMIRIM', 'CASTELO',
      'DIVINO DE SAO LOURENCO', 'DORES DO RIO PRETO', 'GUACUI',
      'IBITIRAMA', 'ICONHA', 'IRUPI', 'ITAPEMIRIM', 'IUNA',
      'JERONIMO MONTEIRO', 'MARATAIZES', 'MIMOSO DO SUL', 'MUNIZ FREIRE',
      'MUQUI', 'PIUMA', 'PRESIDENTE KENNEDY', 'RIO NOVO DO SUL',
      'SAO JOSE DO CALCADO', 'VARGEM ALTA']


    ############################################################################

    # --------- CRIANDO DF_CALENDAR_NEW(CASOS NOVOS) ---------
    #df_calendar_new -> filtrar pacientes com covid confirmados; groupby(Municipio, DataDiagnostico); contar ocorrencias
    df_calendar_new = df[df['Classificacao']=='Confirmados'].groupby(['Municipio','DataDiagnostico'])['DataCadastro'].size().reset_index(name='count_new')

    #renomendo coluna DataDiagnostico
    df_calendar_new.rename(columns={'DataDiagnostico': 'date'}, inplace=True)

    #transformando o dtype na coluna 'date' para datetime
    df_calendar_new['date'] = df_calendar_new['date'].astype('datetime64[ns]')

    # --------- CRIANDO DF_CALENDAR_CLOSED(CASOS FECHADOS) ---------
    #df_calendar_closed -> filtrar pacientes com covid confirmados; groupby(Municipio, DataEncerramento); contar ocorrencias
    df_calendar_closed = df[df['Classificacao']=='Confirmados'].groupby(['Municipio','DataEncerramento'])['DataCadastro'].size().reset_index(name='count_closed')

    #renomendo coluna DataEncerramento
    df_calendar_closed.rename(columns={'DataEncerramento': 'date'}, inplace=True)

    #transformando o dtype na coluna 'date' para datetime
    df_calendar_closed['date'] = df_calendar_closed['date'].astype('datetime64[ns]')

    # --------- CRIANDO DF_CALENDAR_OBITOS ---------
    df_calendar_obitos = df[(df['Classificacao']=='Confirmados') & (df['Evolucao'] == 'Óbito pelo COVID-19')].groupby(['Municipio','DataEncerramento'])['DataCadastro'].size().reset_index(name='count_obito')

    #renomendo coluna DataEncerramento
    df_calendar_obitos.rename(columns={'DataEncerramento': 'date'}, inplace=True)

    #transformando o dtype na coluna 'date' para datetime
    df_calendar_obitos['date'] = df_calendar_obitos['date'].astype('datetime64[ns]')

    # --------- MERGE ENTRE OS DFs CRIADOS ---------
    df_calendar_temp = pd.merge(df_calendar_new, df_calendar_closed, how='outer', left_on=['Municipio', 'date'], right_on=['Municipio', 'date'])
    df_calendar = pd.merge(df_calendar_temp, df_calendar_obitos, how='outer', left_on=['Municipio', 'date'], right_on=['Municipio', 'date'])

    # --------- FILTRO DE DATAS ---------
    #filtrando por período entre 2020 e 2021 (extrair dados com datas erradas)
    df_calendar = df_calendar[(df_calendar['date'].dt.year>=2020) & (df_calendar['date'].dt.year<2022)]

    # --------- DF DE DATAS AUXILIAR ---------
    #criando df_date_aux para preencher o df_calendar com todas as datas do períodom, e fazendo o merge com o df_calendar por cidade
    municipios = df_calendar['Municipio'].unique()

    for i in municipios:
        delta = df_calendar[df_calendar['Municipio']==i]['date'].max() - df_calendar[df_calendar['Municipio']==i]['date'].min()
        df_date_aux = pd.DataFrame({'Municipio':i, 'date': pd.date_range(df_calendar[df_calendar['Municipio']==i]['date'].min(), periods=delta.days).tolist()})
        df_calendar = pd.merge(df_calendar, df_date_aux, how='outer', left_on=['Municipio', 'date'], right_on=['Municipio', 'date'])

    # --------- AJUSTANDO DF_CALENDAR ---------
    #organizando por cidade/data
    df_calendar = df_calendar.sort_values(["Municipio", "date"]).reset_index()
    df_calendar = df_calendar.drop('index', axis=1)

    # --------- DEALING WITH NAN ---------
    #preenchendo Nan com zero(0)
    for i in ['count_new', 'count_closed', 'count_obito']:
        df_calendar[i] = df_calendar[i].fillna(0)

    # --------- FEATURE ENGINEERING ---------
    #criando coluna acumulado por cidade
    municipios = df_calendar['Municipio'].unique()
    acum = []
    for idx, i in enumerate(df_calendar['date']):
        if df_calendar['Municipio'].iloc[idx] == df_calendar['Municipio'].iloc[idx-1]:
            try:
                acum.append(acum[idx-1] + df_calendar['count_new'].iloc[idx] - df_calendar['count_closed'].iloc[idx])
            except:
                acum.append(df_calendar['count_new'].iloc[idx] - df_calendar['count_closed'].iloc[idx])
        else:
            acum.append(0 + df_calendar['count_new'].iloc[idx] - df_calendar['count_closed'].iloc[idx])

    df_calendar['acum'] = acum

    #criando coluna casos recuperados por cidade
    df_calendar['recup'] = df_calendar['count_closed'] - df_calendar['count_obito']


    #criando colunas de dia/semana/dia_da_semana/mes/ano
    df_calendar['day'] = [i.day for i in df_calendar['date']]
    df_calendar['week'] = [i.week for i in df_calendar['date']]
    df_calendar['weekday'] = [i.weekday() for i in df_calendar['date']]
    df_calendar['month'] = [i.month for i in df_calendar['date']]
    df_calendar['year'] = [i.year for i in df_calendar['date']]


    #---------- CRIANDO COLUNAS PARA SEREM UTILIZADAS NO PLOT DE RESUMO ----------
    cidades = df_calendar['Municipio'].unique()

    #--- TOTAL CASOS FATAIS ACUMULADOS POR DIA POR CIDADE ---
    fatais = []
    for i in cidades:
        for j in range(len(df_calendar[df_calendar['Municipio']==i])):
            if j == 0:
                total = df_calendar[df_calendar['Municipio']==i]['count_obito'].iloc[j]
                fatais.append(total)
            else:
                total = total+df_calendar[df_calendar['Municipio']==i]['count_obito'].iloc[j]
                fatais.append(total)

    df_calendar['fatais'] = fatais


    #--- TOTAL CASOS CONFIRMADOS ACUMULADOS POR DIA POR CIDADE ---
    confirmados = []
    for i in cidades:
        for j in range(len(df_calendar[df_calendar['Municipio']==i])):
            if j == 0:
                total = df_calendar[df_calendar['Municipio']==i]['count_new'].iloc[j]
                confirmados.append(total)
            else:
                total = total+df_calendar[df_calendar['Municipio']==i]['count_new'].iloc[j]
                confirmados.append(total)

    df_calendar['confirmados'] = confirmados

    #--- TOTAL CASOS RECUPERADOS ACUMULADOS POR DIA POR CIDADE ---
    recuperados = []
    for i in cidades:
        for j in range(len(df_calendar[df_calendar['Municipio']==i])):
            if j == 0:
                total = df_calendar[df_calendar['Municipio']==i]['recup'].iloc[j]
                recuperados.append(total)
            else:
                total = total+df_calendar[df_calendar['Municipio']==i]['recup'].iloc[j]
                recuperados.append(total)

    df_calendar['recuperados'] = recuperados
    
    df_calendar.to_parquet('MICRODADOS_tratado.parquet')